### PBL2_5
- 고객 이탈 여부 예측을 위한 딥러닝 분류 모델 설계

- 케라스 모델 구성(Input --> Dense --> Dropout --> Dense)
- sklearn metrics로 평가 지표 계산


- 전처리
    - ContractType --> 원 핫 인코딩
    - StandaraScaler로 정규화

- 모델
    - Dense 레이어 기반 MLP
    - 시그모이드 출력, Binary Crossentropy 손실 함수
    - 클래스 불균형을 고려한 가중치 적용(예:class_weights = {0:1.0, 1:2.0})

- 평가: 평가 지표 및 분류 보고서 출력

- 과적합 방지 및 시그모이드 구조 반영하기
- 성능 우수 여부 판단하기(평가 지표)
- 이진 분류 전처리 처리 능력
    - 클래스 불균형 처리를 위한 가중치 적용 또는 분리 전략 여부
- 보고서 구성 및 혼동행렬 해석



In [33]:
#라이브러리 로드
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report


In [20]:
''' 
필요한 기능
1. 원 핫 인코딩 및StandardScaler 정규화 기능
2. 모델을 통한 학습 및 결과 출력(keras, Dense Layer 기반 MLP(Dropout 포함))
3. 평가 지표 계산을 통해 결과 분석
'''

#데이터 생성
df = pd.read_csv('customer_data_balanced.csv')


In [21]:
#데이터 전처리
#특성과 타깃을 분리
#특성: X
X = df.drop('IsChurn', axis=1) 
#타깃: y
y = df['IsChurn']

#원 핫 인코딩
df = pd.get_dummies(df, columns=['ContractType'])


In [22]:
#데이터 분할
X_train, X_temp, y_train, y_temp = train_test_split(
    df, 
    y, 
    test_size=0.3, 
    random_state=42)


In [23]:
#나머지 데이터(30%)를 검증 데이터와 테스트 데이터로 분할
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, 
    y_temp,
    test_size=0.5,
    random_state=42)


In [26]:
#데이터 전처리
#데이터 정규화
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)


In [27]:
#데이터 크기 확인
print('train data size: ', X_train.shape, y_train.shape)
print('validatioin data size: ', X_val.shape, y_val.shape)
print('test data size: ', X_test.shape, y_test.shape)

train data size:  (1400, 8) (1400,)
validatioin data size:  (300, 8) (300,)
test data size:  (300, 8) (300,)


In [32]:
#모델 설계
#입력층, 은닉층, 출력층으로 구성된 이진 분류 모델

model = tf.keras.Sequential([
    #입력층
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    #첫 번째 은닉층
    tf.keras.layers.Dense(64, activation='relu'),
    #과적합 방지를 위한 Dropout
    tf.keras.layers.Dropout(0.5),
    #두 번째 은닉층
    tf.keras.layers.Dense(32, activation='relu'),
    #출력층(Sigmoid)
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [34]:
#모델 컴파일
#손실함수: Binary Crossentropy
#최적화 알고리즘: Adam
#평가지표: Accuracy,F1-Score, Confusion Matrics, Classification_report
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
#Early Stopping 설정. 0.000001보다 작게 개선되면 조기 종료
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=0.000001, restore_best_weights=True)


In [ ]:
#모델 훈련
history = model.fit(X_train, 
                    y_train,
                    validation_data=(X_val, y_val),
                    epochs=50,
                    batch_size=32,
                    callbacks=[early_stopping])


Epoch 1/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 1.8060e-09 - val_accuracy: 1.0000 - val_loss: 1.4119e-14
Epoch 2/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 4.1095e-09 - val_accuracy: 1.0000 - val_loss: 1.4013e-14
Epoch 3/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 3.9727e-09 - val_accuracy: 1.0000 - val_loss: 1.3958e-14
Epoch 4/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 2.2907e-07 - val_accuracy: 1.0000 - val_loss: 1.3653e-14
Epoch 5/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 9.6546e-07 - val_accuracy: 1.0000 - val_loss: 4.4997e-15
Epoch 6/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 1.1929e-09 - val_accuracy: 1.0000 - val_loss: 4.1131e-15


In [48]:
#모델 평가
#테스트 데이터로 성능 평가
test_loss, test_accuracy = model.evaluate(X_test, y_test)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 1.5594e-14 


In [53]:
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

Test Loss: 0.0000, Test Accuracy: 1.0000


In [54]:
#예측
predictions = model.predict(X_test)
predicted_classes = (predictions > 0.5).astype(int) #0.5 기준으로 클래스 결정



10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


In [56]:
#분류 보고서 출력
print('Classicication Report: \n', classification_report(y_test, predicted_classes))


Classicication Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       193
           1       1.00      1.00      1.00       107

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300



In [ ]:
#혼동 행렬 출력
''' 
모든 데이터들을 정확하게 분류. TP, TN 100%.
'''
print('Confusion Matrix: ')
print(confusion_matrix(y_test, predicted_classes))

Confusion Matrix: 
[[193   0]
 [  0 107]]
